In [28]:
import pandas as pd 
import numpy as np 
import os 

In [ ]:
file = "/Users/fareyaikram/Workspace/dialogue-kt-main/fareya/raw_data/LEVI_tutoring_dataset_round1.csv"
df = pd.read_csv(file)
my_cols = df.columns[4:-1]
print(len(my_cols))
print(my_cols)

22
Index(['useraccount_id', 'Success', 'confirmatory feedback',
       'negative feedback', 'correcting', 'giving instruction',
       'giving explanation', 'providing further references', 'questioning',
       'asking for elaboration', 'praising and encouraging',
       'managing frustration', 'managing discussions', 'giving answers',
       'encouraging peer tutoring', 'guiding peer tutoring',
       'acknowledging tutor issue', 'irrelevant statement',
       'computational skill', 'linguistic knowledge', 'conceptual knowledge',
       'strategic knowledge'],
      dtype='object')


In [30]:
# find the unique ids in the dataframe
print(len(df['id'].unique()))

2512


In [31]:
df_no_duplicates = df.drop_duplicates(['content'])
df_no_duplicates['processed_text'] = df_no_duplicates.apply(lambda x: x['content'].split("*document")[0], axis=1) 
print(df_no_duplicates['processed_text'].head(40))

AttributeError: 'float' object has no attribute 'split'

In [7]:
#write a function that underscore between the words in the column names
def underscore_column_names(columns):
    new_columns = []
    for column in columns:
        column = column.replace(" ", "_")
        new_columns.append(column.lower())
    
    return str(new_columns)

In [8]:
df_no_duplicates.columns

Index(['id', 'id2', 'content', 'Success', 'confirmatory feedback',
       'negative feedback', 'correcting', 'giving instruction',
       'giving explanation', 'providing further references', 'questioning',
       'asking for elaboration', 'praising and encouraging',
       'managing frustration', 'managing discussions', 'giving answers',
       'encouraging peer tutoring', 'guiding peer tutoring',
       'acknowledging tutor issue', 'other', 'irrelevant statement',
       'computational skill', 'linguistic knowledge', 'conceptual knowledge',
       'strategic knowledge', 'affective control', 'processed_text'],
      dtype='object')

In [9]:
df_no_duplicates['list_of_labels'] = df_no_duplicates.apply(lambda x: underscore_column_names([col for col in my_cols if x[col] == 1]), axis=1)

/var/folders/1j/s7v97cmj2_b4j84vjkcbfkl00000gn/T/ipykernel_3598/460016489.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates['list_of_labels'] = df_no_duplicates.apply(lambda x: underscore_column_names([col for col in my_cols if x[col] == 1]), axis=1)


In [10]:
# if proccessed_text contains text that starts with 'u' and is followed by a number and a colon, then replace it with 'Student'
# if proccessed_text contains text that starts with 'p' and is followed by a number and a colon, then replace it with 'Tutor'

df_no_duplicates['processed_text'] = df_no_duplicates['processed_text'].replace(to_replace=r'^[u]\d+:', value='Student:', regex=True)
df_no_duplicates['processed_text'] = df_no_duplicates['processed_text'].replace(to_replace=r'^[p]\d+:', value='Tutor:', regex=True)
df_no_duplicates['processed_text'] = df_no_duplicates['processed_text'].replace(to_replace=r'^[e]\d+:', value='Teacher:', regex=True)


/var/folders/1j/s7v97cmj2_b4j84vjkcbfkl00000gn/T/ipykernel_3598/2515613975.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates['processed_text'] = df_no_duplicates['processed_text'].replace(to_replace=r'^[u]\d+:', value='Student:', regex=True)
/var/folders/1j/s7v97cmj2_b4j84vjkcbfkl00000gn/T/ipykernel_3598/2515613975.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates['processed_text'] = df_no_duplicates['processed_text'].replace(to_replace=r'^[p]\d+:', value='Tutor:', rege

In [11]:
def is_student(text):
    return text.startswith('Student')
df_no_duplicates['is_tutor'] = df_no_duplicates.apply(lambda x: not is_student(x['processed_text']), axis=1)


/var/folders/1j/s7v97cmj2_b4j84vjkcbfkl00000gn/T/ipykernel_3598/326357659.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates['is_tutor'] = df_no_duplicates.apply(lambda x: not is_student(x['processed_text']), axis=1)


In [ ]:
df_no_duplicates[['processed_text', 'id', 'id2', 'list_of_labels', 'is_tutor', 'Success']].to_json("processed_florida_data_2.jsonl", orient= 'records',index=False, lines=True)

In [26]:
import json 
from collections import defaultdict

def read_jsonl(data_path):
    with open(data_path) as f:
        return [json.loads(line) for line in f]

def group_data_by_id(data):
    # add all ids to a list
    grouped_data = defaultdict(list)
    for entry in data:
        grouped_data[entry["id"]].append(entry)
    
    # remove duplicates 
    # for key in grouped_data.keys():
    #     grouped_data[key] = pd.DataFrame(grouped_data[key]).drop_duplicates(subset=['teacher_move', 'student_move']).to_dict('records')
    return grouped_data

def split_train_val(grouped_data):
    train_data, val_data = train_test_split(list(grouped_data.values()), test_size=0.2, random_state=RANDOM_SEED)
    return train_data, val_data

def get_test_formatted(grouped_data):
    test_data = list(grouped_data.values())
    return test_data

def format_dialogue_anation(data, label_type = "teacher_move_type", add_prev_labels = True): 
    formatted_outputs = []
    labels = []
    ids = [] 
    for i, conversation in enumerate(data):
        dialogue = []
        tutor_labels = []
        is_tutor_list = [] 
        correctness = [] 
        for j, turn in enumerate(conversation): 
            dialogue.append(turn['processed_text'])
            if turn["Success"]:
                correctness.append("true") 
            else: correctness.append("true") 
            if turn['is_tutor']: 
                tutor_labels.append(turn['list_of_labels'])
                is_tutor_list.append(True)
            else: 
                tutor_labels.append(None)
                is_tutor_list.append(False)

        final_dialogue = "[BEGIN DIALOGUE]"
        for k in range(len(conversation)): 
            if k>0 and add_prev_labels:
                prepend_text = "" if not is_tutor_list[k-1] else str(tutor_labels[k-1])
            else: 
                prepend_text = ""
            final_dialogue = final_dialogue + prepend_text + "\n" + dialogue[k]
        
            if is_tutor_list[k] and label_type == "teacher_move_type":
                formatted_outputs.append(final_dialogue + "\n[END DIALOGUE]")
                labels.append(tutor_labels[k])
                ids.append(str(conversation[k]["id"])+str(conversation[k]["id2"]))
            
            if is_tutor_list[k] and label_type == "final_correctness":
                formatted_outputs.append(final_dialogue + "\n[END DIALOGUE]")
                labels.append(correctness[k])
                ids.append(str(conversation[k]["id"])+str(conversation[k]["order"]))

            if k < len(conversation) - 1: 
                if is_tutor_list[k+1] and label_type == "future_teacher_move_type":
                    formatted_outputs.append(final_dialogue + "\n[END DIALOGUE]")
                    labels.append(tutor_labels[k+1])
                    ids.append(str(conversation[k]["id"])+str(conversation[k]["order"]))
    final_formatted_outputs = []
    final_labels = []
    final_ids = [] 

    for i in range(len(formatted_outputs)): 
        if not(labels[i] == "[]"):
            final_formatted_outputs.append(formatted_outputs[i])
            final_labels.append(labels[i])
            final_ids.append(ids[i])
    for i in range(10):
        print(final_formatted_outputs[i])
        print(final_labels[i])
    return final_formatted_outputs, final_labels, final_ids 

In [ ]:
data = read_jsonl("processed_florida_data.jsonl")
grouped_data = group_data_by_id(data)
train_data, val_data = split_train_val(grouped_data)
# write the train and val data to jsonl files
with open("anation_train_data.jsonl", "w") as f:
    for entry in train_data:
        for i in range(len(entry)):
            f.write(json.dumps(entry[i]) + "\n")
with open("anation_val_data.jsonl", "w") as f:
    for entry in val_data:
        for i in range(len(entry)):
            f.write(json.dumps(entry[i]) + "\n")
print("Grouped data: ",len(grouped_data.keys()))
print("Length of train data: ", len(train_data))
print("Length of test data: ", len(test_data))

In [27]:
DATA_PATH = "/Users/fareyaikram/Workspace/dialogue-kt-main/fareya/processed_florida_data_1.jsonl"
output_file = "/Users/fareyaikram/Workspace/dialogue-kt-main/fareya/sample_output.jsonl"
data = read_jsonl(DATA_PATH)
grouped_data = group_data_by_id(data)
test_data = get_test_formatted(grouped_data)
formatted_outputs, labels, ids = format_dialogue_anation(test_data)

with open(output_file, 'w') as f:
    for formatted_output, label, id_ in zip(formatted_outputs, labels, ids):
        json_line = {
            "formatted_output": formatted_output,
            "label": label,
            "id": id_
        }
        f.write(json.dumps(json_line) + '\n')

print(f"Output written to {output_file}")

[BEGIN DIALOGUE]
Student: i finally found out the answer
it was 96 
Tutor: Yeah! 
[END DIALOGUE]
['confirmatory_feedback']
[BEGIN DIALOGUE]
Student: i finally found out the answer
it was 96 
Tutor: Yeah! ['confirmatory_feedback']
Student: i guess every single question then watch the video of how you do it 
Student: what is a thread 
Teacher: great! a thread is all the posts of the same topic
**post closed** 
[END DIALOGUE]
['giving_explanation', 'praising_and_encouraging']
[BEGIN DIALOGUE]
Student: i finally found out the answer
it was 96 
Tutor: Yeah! ['confirmatory_feedback']
Student: i guess every single question then watch the video of how you do it 
Student: what is a thread 
Teacher: great! a thread is all the posts of the same topic
**post closed** ['giving_explanation', 'praising_and_encouraging']
Student: what does **post closed** mean 
Tutor: Don't post after her  
[END DIALOGUE]
['managing_discussions']
[BEGIN DIALOGUE]
Student: i finally found out the answer
it was 96 
Tuto